In [1]:
import networkx as nx

# Створення орієнтованого графа
G = nx.DiGraph()

# Додаємо ребра з пропускною здатністю
edges = [
    ("Термінал 1", "Склад 1", 25),
    ("Термінал 1", "Склад 2", 20),
    ("Термінал 1", "Склад 3", 15),
    ("Термінал 2", "Склад 3", 15),
    ("Термінал 2", "Склад 4", 30),
    ("Термінал 2", "Склад 2", 10),
    ("Склад 1", "Магазин 1", 15),
    ("Склад 1", "Магазин 2", 10),
    ("Склад 1", "Магазин 3", 20),
    ("Склад 2", "Магазин 4", 15),
    ("Склад 2", "Магазин 5", 10),
    ("Склад 2", "Магазин 6", 25),
    ("Склад 3", "Магазин 7", 20),
    ("Склад 3", "Магазин 8", 15),
    ("Склад 3", "Магазин 9", 10),
    ("Склад 4", "Магазин 10", 20),
    ("Склад 4", "Магазин 11", 10),
    ("Склад 4", "Магазин 12", 15),
    ("Склад 4", "Магазин 13", 5),
    ("Склад 4", "Магазин 14", 10),
]

# Додаємо ребра до графа
for u, v, capacity in edges:
    G.add_edge(u, v, capacity=capacity)

# Додаємо джерело і стік
G.add_edge("Джерело", "Термінал 1", capacity=float("inf"))
G.add_edge("Джерело", "Термінал 2", capacity=float("inf"))
for i in range(1, 15):
    G.add_edge(f"Магазин {i}", "Стік", capacity=float("inf"))

# Обчислення максимального потоку
flow_value, flow_dict = nx.maximum_flow(G, "Джерело", "Стік")

# Вивід результату
print("Загальний максимальний потік:", flow_value)
print("\nФактичні потоки від терміналів до магазинів:")
for terminal in ["Термінал 1", "Термінал 2"]:
    for warehouse, warehouse_flow in flow_dict[terminal].items():
        for shop, shop_flow in flow_dict.get(warehouse, {}).items():
            if shop.startswith("Магазин") and shop_flow > 0:
                print(f"{terminal} → {shop}: {shop_flow}")


Загальний максимальний потік: 115

Фактичні потоки від терміналів до магазинів:
Термінал 1 → Магазин 1: 15
Термінал 1 → Магазин 2: 10
Термінал 1 → Магазин 4: 15
Термінал 1 → Магазин 5: 10
Термінал 1 → Магазин 6: 5
Термінал 1 → Магазин 7: 20
Термінал 1 → Магазин 8: 10
Термінал 2 → Магазин 7: 20
Термінал 2 → Магазин 8: 10
Термінал 2 → Магазин 10: 20
Термінал 2 → Магазин 11: 10
Термінал 2 → Магазин 4: 15
Термінал 2 → Магазин 5: 10
Термінал 2 → Магазин 6: 5


In [2]:
import pandas as pd

# Сформуємо список результатів із потоку
rows = []
for terminal in ["Термінал 1", "Термінал 2"]:
    for warehouse, warehouse_flow in flow_dict[terminal].items():
        for shop, shop_flow in flow_dict.get(warehouse, {}).items():
            if shop.startswith("Магазин") and shop_flow > 0:
                rows.append({
                    "Термінал": terminal,
                    "Магазин": shop,
                    "Фактичний Потік (одиниць)": shop_flow
                })

# Побудова таблиці
df = pd.DataFrame(rows)
df = df.sort_values(by=["Термінал", "Магазин"])
df.reset_index(drop=True, inplace=True)

# Виведення
df.style.set_caption("Таблиця фактичних потоків від терміналів до магазинів")


,Термінал,Магазин,Фактичний Потік (одиниць)
0,Термінал 1,Магазин 1,15
1,Термінал 1,Магазин 2,10
2,Термінал 1,Магазин 4,15
3,Термінал 1,Магазин 5,10
4,Термінал 1,Магазин 6,5
5,Термінал 1,Магазин 7,20
6,Термінал 1,Магазин 8,10
7,Термінал 2,Магазин 10,20
8,Термінал 2,Магазин 11,10
9,Термінал 2,Магазин 4,15


Запитання 1. Які термінали забезпечують найбільший потік товарів?

In [3]:
# Підрахунок сумарного потоку з кожного термінала
from collections import defaultdict

terminal_flow_total = defaultdict(int)

for terminal in ["Термінал 1", "Термінал 2"]:
    for warehouse, w_flow in flow_dict[terminal].items():
        for shop, flow in flow_dict.get(warehouse, {}).items():
            if shop.startswith("Магазин"):
                terminal_flow_total[terminal] += flow

# Виведення результату
for terminal, total in terminal_flow_total.items():
    print(f"{terminal}: {total} одиниць")

max_terminal = max(terminal_flow_total, key=terminal_flow_total.get)
print(f"\n🔹 Найбільший потік забезпечує: {max_terminal}")


Термінал 1: 85 одиниць
Термінал 2: 90 одиниць

🔹 Найбільший потік забезпечує: Термінал 2


Запитання 2. Які маршрути мають найменшу пропускну здатність?

In [4]:
# Пошук ребер з найменшою пропускною здатністю
min_capacity = float('inf')
min_capacity_edges = []

for u, v, data in G.edges(data=True):
    cap = data.get("capacity", float('inf'))
    if cap != float('inf'):
        if cap < min_capacity:
            min_capacity = cap
            min_capacity_edges = [(u, v)]
        elif cap == min_capacity:
            min_capacity_edges.append((u, v))

print(f"Мінімальна пропускна здатність: {min_capacity} одиниць")
print("Маршрути з такою пропускною здатністю:")
for u, v in min_capacity_edges:
    print(f"➡️ {u} → {v}")


Мінімальна пропускна здатність: 5 одиниць
Маршрути з такою пропускною здатністю:
➡️ Склад 4 → Магазин 13


Запитання 3. Які магазини отримали найменше товарів?

In [5]:
# Сумарний потік до кожного магазину
shop_flow = defaultdict(int)

for terminal in ["Термінал 1", "Термінал 2"]:
    for warehouse in flow_dict[terminal]:
        for shop, flow in flow_dict.get(warehouse, {}).items():
            if shop.startswith("Магазин"):
                shop_flow[shop] += flow

# Виведемо топ 5 магазинів з найменшим потоком
sorted_shops = sorted(shop_flow.items(), key=lambda x: x[1])
print("🏬 Магазини, що отримали найменше товарів:")
for shop, value in sorted_shops[:5]:
    print(f"{shop}: {value} одиниць")


🏬 Магазини, що отримали найменше товарів:
Магазин 3: 0 одиниць
Магазин 9: 0 одиниць
Магазин 12: 0 одиниць
Магазин 13: 0 одиниць
Магазин 14: 0 одиниць


Запитання 4. Чи є вузькі місця, які можна усунути?

In [6]:
# Вузькі місця = маршрути з найменшою пропускною здатністю та нульовим або малим потоком
narrow_spots = []

for u, v, data in G.edges(data=True):
    cap = data.get("capacity", float('inf'))
    flow = flow_dict.get(u, {}).get(v, 0)
    if cap != float('inf') and flow < cap // 2:
        narrow_spots.append((u, v, flow, cap))

print("🚧 Потенційні вузькі місця:")
for u, v, flow, cap in narrow_spots:
    print(f"{u} → {v}: потік {flow} з {cap} (використано {flow / cap:.0%})")


🚧 Потенційні вузькі місця:
Склад 1 → Магазин 3: потік 0 з 20 (використано 0%)
Склад 2 → Магазин 6: потік 5 з 25 (використано 20%)
Склад 3 → Магазин 9: потік 0 з 10 (використано 0%)
Склад 4 → Магазин 12: потік 0 з 15 (використано 0%)
Склад 4 → Магазин 13: потік 0 з 5 (використано 0%)
Склад 4 → Магазин 14: потік 0 з 10 (використано 0%)
